In [1]:
import pandas as pd

import pickle
from tqdm import tqdm

In [3]:
test_pred = pickle.load(open("model_sep14_pubmedbert_5lr_5epochs/pubmedbertpreds.pkl", "rb"))

In [6]:
label_map = {'ACTIVATOR': 0, 'AGONIST': 1, 'AGONIST-ACTIVATOR': 2, 'AGONIST-INHIBITOR': 3, 'ANTAGONIST': 4,
 'DIRECT-REGULATOR': 5, 'INDIRECT-DOWNREGULATOR': 6, 'INDIRECT-UPREGULATOR': 7, 'INHIBITOR': 8,
 'PART-OF': 9, 'PRODUCT-OF': 10, 'SUBSTRATE': 11, 'SUBSTRATE_PRODUCT-OF': 12, 'false': 13}

In [7]:
label_map_index = { label_map[i]:i for i in label_map}


In [8]:
test_pred_map = []
for pred in test_pred:
    test_pred_map.append(label_map_index[pred])

In [9]:
cols = ['pmid_vals', 'sentence']
df_test = pd.read_csv("processed_data/test.tsv", sep='\t', names=cols)

In [10]:
df_test[['PMID','arg1', 'arg2']] = df_test.pmid_vals.str.split(".",expand=True)

In [11]:
df_test['label'] = test_pred_map 

In [12]:
df_tosubmit = df_test[df_test['label'] != 'false']

In [14]:
df_tosubmit.shape

(66416, 6)

In [15]:
pmid = df_tosubmit.PMID.tolist()
arg1 = df_tosubmit.arg1.tolist()
arg2 = df_tosubmit.arg2.tolist()

In [16]:
len(pmid)

66416

In [17]:
file = "drugprot-gs-training-development/test-background/test_background_entities.tsv"
text = open(file).read().splitlines()

data = [x.split("\t") for x in text]
df_test_entities = pd.DataFrame(data, columns= ["PMID", "arg", "entity", "start", "end", "name"])


In [18]:
new_arg1 = []
new_arg2 = []
for i in tqdm(range(len(pmid))):
#for i in range(100):
    check_entity = df_test_entities[(df_test_entities['PMID'] == pmid[i]) & (df_test_entities['arg'] == arg1[i])]['entity']
    #print(type(check_entity))
    if check_entity.values == 'GENE':
        new_arg1.append(arg2[i])
        new_arg2.append(arg1[i])
    else:
        new_arg1.append(arg1[i])
        new_arg2.append(arg2[i])

100%|██████████| 66416/66416 [40:23<00:00, 27.40it/s]


In [21]:
df_tosubmit.head()

,pmid_vals,sentence,PMID,arg1,arg2,label,new_arg1,new_arg2
6,32733640.T17.T30,"In moderately injured P19 neuronal cells, conc...",32733640,T17,T30,INHIBITOR,T17,T30
7,32733640.T18.T31,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T31,INDIRECT-DOWNREGULATOR,T18,T31
9,32733640.T18.T33,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T33,INDIRECT-DOWNREGULATOR,T18,T33
10,32733640.T18.T34,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T34,INDIRECT-UPREGULATOR,T18,T34
11,32733640.T18.T35,Western blot analysis revealed that @CHEMICAL$...,32733640,T18,T35,INDIRECT-UPREGULATOR,T18,T35


In [22]:
df_tosubmit["new_arg1"] = new_arg1
df_tosubmit["new_arg2"] = new_arg2

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [23]:
df_tosubmit["new_arg1"] = "Arg1:"+ df_tosubmit["new_arg1"]
df_tosubmit["new_arg2"] = "Arg2:"+ df_tosubmit["new_arg2"]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [24]:
df_final = df_tosubmit[['PMID','label', 'new_arg1','new_arg2']]

In [26]:
df_final.to_csv("test_submission_sep15/pubmertbert.tsv", sep="\t", index=False, header=None)